In [ ]:
import numpy as np

class ConvLayer:
    def __init__(self, num_filters, filter_size):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.filters = np.random.randn(num_filters, filter_size, filter_size) / filter_size**2

    def iterate_regions(self, image):
        h, w = image.shape
        for i in range(h - self.filter_size + 1):
            for j in range(w - self.filter_size + 1):
                yield image[i:(i + self.filter_size), j:(j + self.filter_size)], i, j

    def forward(self, input):
        self.last_input = input
        h, w = input.shape
        output = np.zeros((h - self.filter_size + 1, w - self.filter_size + 1, self.num_filters))

        for region, i, j in self.iterate_regions(input):
            output[i, j] = np.sum(region * self.filters, axis=(1, 2))
        return output

    def backprop(self, d_L_d_out, learning_rate):
        d_L_d_filters = np.zeros(self.filters.shape)
        for region, i, j in self.iterate_regions(self.last_input):
            for f in range(self.num_filters):
                d_L_d_filters[f] += d_L_d_out[i, j, f] * region

        self.filters -= learning_rate * d_L_d_filters
        return None

class MaxPoolLayer:
    def __init__(self, pool_size):
        self.pool_size = pool_size

    def iterate_regions(self, image):
        h, w, _ = image.shape
        new_h = h // self.pool_size
        new_w = w // self.pool_size

        for i in range(new_h):
            for j in range(new_w):
                yield image[(i * self.pool_size):(i * self.pool_size + self.pool_size), (j * self.pool_size):(j * self.pool_size + self.pool_size)], i, j

    def forward(self, input):
        self.last_input = input
        h, w, num_filters = input.shape
        output = np.zeros((h // self.pool_size, w // self.pool_size, num_filters))

        for region, i, j in self.iterate_regions(input):
            output[i, j] = np.amax(region, axis=(0, 1))
        return output

    def backprop(self, d_L_d_out):
        d_L_d_input = np.zeros(self.last_input.shape)

        for region, i, j in self.iterate_regions(self.last_input):
            h, w, f = region.shape
            amax = np.amax(region, axis=(0, 1))

            for i2 in range(h):
                for j2 in range(w):
                    for f2 in range(f):
                        if region[i2, j2, f2] == amax[f2]:
                            d_L_d_input[i * self.pool_size + i2, j * self.pool_size + j2, f2] = d_L_d_out[i, j, f2]
        return d_L_d_input

class SoftmaxLayer:
    def __init__(self, input_len, nodes):
        self.weights = np.random.randn(input_len, nodes) / input_len
        self.biases = np.zeros(nodes)

    def forward(self, input):
        self.last_input_shape = input.shape
        input = input.flatten()
        self.last_input = input
        input_len, nodes = self.weights.shape

        totals = np.dot(input, self.weights) + self.biases
        self.last_totals = totals

        exp = np.exp(totals)
        return exp / np.sum(exp, axis=0)

    def backprop(self, d_L_d_out, learning_rate):
        for i, gradient in enumerate(d_L_d_out):
            if gradient == 0:
                continue

            t_exp = np.exp(self.last_totals)
            S = np.sum(t_exp)

            d_out_d_t = -t_exp[i] * t_exp / (S ** 2)
            d_out_d_t[i] = t_exp[i] * (S - t_exp[i]) / (S ** 2)

            d_t_d_w = self.last_input
            d_t_d_b = 1
            d_t_d_inputs = self.weights

            d_L_d_t = gradient * d_out_d_t

            d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
            d_L_d_b = d_L_d_t * d_t_d_b
            d_L_d_inputs = d_t_d_inputs @ d_L_d_t

            self.weights -= learning_rate * d_L_d_w
            self.biases -= learning_rate * d_L_d_b
            return d_L_d_inputs.reshape(self.last_input_shape)

class CNN:
    def __init__(self):
        self.conv = ConvLayer(8, 3)
        self.pool = MaxPoolLayer(2)
        self.softmax = SoftmaxLayer(13 * 13 * 8, 10)

    def forward(self, image):
        output = self.conv.forward(image)
        output = self.pool.forward(output)
        output = self.softmax.forward(output)
        return output

    def train(self, image, label, lr=0.005):
        output = self.forward(image)
        loss = -np.log(output[label])
        acc = 1 if np.argmax(output) == label else 0

        gradient = np.zeros(10)
        gradient[label] = -1 / output[label]

        gradient = self.softmax.backprop(gradient, lr)
        gradient = self.pool.backprop(gradient)
        self.conv.backprop(gradient, lr)

        return loss, acc



In [ ]:
import numpy as np

class RNNLayer:
    def __init__(self, input_size, output_size, hidden_size):
        self.Whh = np.random.randn(hidden_size, hidden_size) / 1000
        self.Wxh = np.random.randn(hidden_size, input_size) / 1000
        self.Why = np.random.randn(output_size, hidden_size) / 1000

        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))

    def forward(self, inputs):
        self.last_inputs = inputs
        self.last_hs = {0: np.zeros((self.Whh.shape[0], 1))}

        for i, x in enumerate(inputs):
            self.last_hs[i + 1] = np.tanh(
                self.Wxh @ x + self.Whh @ self.last_hs[i] + self.bh
            )

        return self.Why @ self.last_hs[len(inputs)] + self.by

    def backprop(self, d_y, learning_rate=2e-2):
        n = len(self.last_inputs)

        d_Why = d_y @ self.last_hs[n].T
        d_by = d_y

        d_Whh = np.zeros(self.Whh.shape)
        d_Wxh = np.zeros(self.Wxh.shape)
        d_bh = np.zeros(self.bh.shape)

        d_h = self.Why.T @ d_y

        for t in reversed(range(n)):
            temp = ((1 - self.last_hs[t + 1] ** 2) * d_h)
            d_bh += temp
            d_Whh += temp @ self.last_hs[t].T
            d_Wxh += temp @ self.last_inputs[t].T

            d_h = self.Whh @ temp

        for d in [d_Wxh, d_Whh, d_Why, d_bh, d_by]:
            np.clip(d, -1, 1, out=d)

        self.Whh -= learning_rate * d_Whh
        self.Wxh -= learning_rate * d_Wxh
        self.Why -= learning_rate * d_Why
        self.bh -= learning_rate * d_bh
        self.by -= learning_rate * d_by

class RNN:
    def __init__(self, input_size, output_size, hidden_size):
        self.rnn_layer = RNNLayer(input_size, output_size, hidden_size)

    def forward(self, inputs):
        return self.rnn_layer.forward(inputs)

    def train(self, inputs, targets, learning_rate=2e-2):
        out = self.forward(inputs)
        loss = np.sum((out - targets) ** 2) / out.shape[0]

        d_L_d_y = 2 * (out - targets) / out.shape[0]
        self.rnn_layer.backprop(d_L_d_y, learning_rate)

        return loss


``
